<a href="https://colab.research.google.com/github/cs-amy/project-codebase/blob/main/Word_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CNN Sliding-Window Model for 3-Letter Word De-Obfuscation**
Stage 2 of MSc Project — Ashraf Muhammed Yusuf

# **1. Colab Environment Setup**

In [1]:
# Install dependencies
!pip install -q tensorflow matplotlib

In [2]:
# Import dependencies
import os, sys, random, itertools, pathlib, math, shutil
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf, os, numpy as np
from pathlib import Path
from glob import glob
from tensorflow.keras import mixed_precision
from google.colab import drive
from tensorflow.keras.callbacks import (ModelCheckpoint, EarlyStopping, ReduceLROnPlateau)
from sklearn.metrics import classification_report, confusion_matrix
from collections import defaultdict
from PIL import Image, ImageDraw, ImageFont
from tqdm import tqdm
import glob

In [ ]:
# 1.3 Mount Drive & define base path
# Mount Drive so you can read datasets and write checkpoints
# Link to dataset:
# https://drive.google.com/drive/folders/1sfNG1PkmTPBe1wOSQXZmfdkvR97Hn9lk?usp=sharing
drive.mount('/content/drive')

In [4]:
# (Optional—but useful) turn on XLA JIT for extra speed
tf.config.optimizer.set_jit(True)

# **2. Data Generation**
This block generates the 'three-letter words' dataset afresh if you do not already have it (You can access it here: https://drive.google.com/drive/folders/1kygA17GiCeCs8qTeDBEndU6TkXnEu-m7?usp=drive_link). It synthesizes three three-letter words from the character dataset (https://drive.google.com/drive/folders/1eUaTNW8zVjTArg0JszbCdCEq0tTdx89n?usp=drive_link).

In [5]:
# paths & constants
BASE_PATH = Path("/content/drive/MyDrive/MScProject")
GLYPH_DIR = Path(f"{BASE_PATH}/data/characters/train")
DATA_ROOT = Path(f"{BASE_PATH}/data/words3")
CKPT_DIR = f"{BASE_PATH}/words3_ckpt_best.keras"
BATCH = 128
IMG_H = IMG_W = 64
IMG_SHAPE = (IMG_H, IMG_W)
PATCH_W = IMG_W // 3
VARIANTS_PER = 5 # per word
EXPECTED_CLASSES = 26**3 # 26³ = 17,576
FINAL_TEST_FRAC = 0.20
SEED = 42
PATCH_W = IMG_W // 3 # 21 when IMG_W = 64

random.seed(SEED)
train_dir = DATA_ROOT / "train"
test_dir  = DATA_ROOT / "test"

In [ ]:
# ================================================================
# 3-Letter Word Dataset Builder  (one-cell, idempotent)
# ------------------------------------------------
# If words3/train & words3/test already contain 17,576 class folders
#   - skips everything (fast guard)
# Else:
#   1. Generates synthetic uppercase dataset (train/ val/ test/)
#   2. Merges val & test into train
#   3. Re-splits 80 / 20 into final train/ and test/
# ------------------------------------------------
# Adjust DATA_ROOT and GLYPH_DIR to your own Drive paths.
# ================================================================

# fast guard
def class_dir_count(p: Path) -> int:
  return sum(1 for q in p.iterdir() if q.is_dir())


if (train_dir.exists() and test_dir.exists() and
  class_dir_count(train_dir) == EXPECTED_CLASSES and
  class_dir_count(test_dir)  == EXPECTED_CLASSES):
  print("words3 dataset already complete – skipping generation.")
  raise SystemExit

# glyph reservoir
letter_pool = {ltr: glob(str(GLYPH_DIR / ltr / "*.png"))
               for ltr in "ABCDEFGHIJKLMNOPQRSTUVWXYZ"}
missing = [l for l,v in letter_pool.items() if not v]
if missing:
    sys.exit(f"Missing glyphs for {missing} in {GLYPH_DIR}")

# obfuscation maps
LEET = {
  'A': ['Α', '4', 'Д', 'Ä', 'Á', 'À', 'Â', '@', 'Δ'],
  'B': ['8', 'β', 'Β', 'В'],
  'C': ['Ç', 'Ć', 'Č', 'С'],
  'D': ['Ð', 'Ď'],
  'E': ['3', 'Σ', 'Έ', 'Ε', 'Е', 'Ë', 'É', 'È', 'Ê'],
  'F': ['Φ', 'Ƒ'],
  'G': ['6', 'Ğ', 'Ģ', 'Γ'],
  'H': ['Η', 'Н'],
  'I': ['1', '|', 'Í', 'Ì', 'Î', 'Ï', 'И'],
  'J': ['Ј'],
  'K': ['Κ', 'К'],
  'L': ['Ι', 'Ł', 'Ĺ', 'Л'],
  'M': ['Μ', 'М'],
  'N': ['Ν', 'Ń', 'Ñ', 'Н'],
  'O': ['0', 'Θ', 'Ο', 'Ө', 'Ø', 'Ö', 'Ó', 'Ò', 'Ô'],
  'P': ['Ρ', 'Р'],
  'Q': ['Φ'],
  'R': ['®', 'Я', 'Ř', 'Ŕ'],
  'S': ['5', '$', 'Ѕ', 'Ś', 'Š'],
  'T': ['Τ', 'Т'],
  'U': ['Υ', 'Ц', 'Ü', 'Ú', 'Ù', 'Û'],
  'V': ['Ѵ', 'V'],
  'W': ['Ω', 'Ѡ', 'Ψ', 'Ш', 'Щ'],
  'X': ['Χ', 'Ж', 'Х'],
  'Y': ['Υ', 'Ү', 'Ý', 'Ÿ'],
  'Z': ['Ζ', 'Ż', 'Ź', 'Ž', 'З', '2']
}
HOMO = {
  'A':'Α',
  'B':'Β',
  'C':'С',
  'E':'Ε',
  'H':'Н',
  'K':'Κ',
  'M':'Μ',
  'O':'О',
  'P':'Р',
  'T':'Τ',
  'X':'Χ',
  'Y':'Υ',
  'Z':'Ζ'
}

# Obfuscation helper
def obfuscate(ch):
  mode = random.choices(("none","leet","homo"), weights=(0.5,0.4,0.1))[0]
  if mode=="leet" and ch in LEET: return random.choice(LEET[ch])
  if mode=="homo" and ch in HOMO: return HOMO[ch]
  return ch

# image composer
def stitch(word, out_path):
  canvas = Image.new("L",(IMG_W,IMG_H),color=255)
  for i,ch in enumerate(word):
    if random.random()<0.7:
      glyph = Image.open(random.choice(letter_pool[ch])).resize((PATCH_W,IMG_H))
    else:
      glyph = Image.new("L",(PATCH_W,IMG_H),color=255)
      ImageDraw.Draw(glyph).text((4,4), obfuscate(ch), fill=0)
    canvas.paste(glyph,(i*PATCH_W,0))
  if random.random()<0.3:
    dx = random.randint(-2,2)
    canvas = canvas.transform(canvas.size, Image.AFFINE, (1,0,dx,0,1,0))
  canvas.save(out_path)

# 1. generate initial train/val/test
if not (DATA_ROOT/"train").exists():
  print("➤ Generating synthetic dataset …")
  words = ["".join(p) for p in itertools.product("ABCDEFGHIJKLMNOPQRSTUVWXYZ", repeat=3)]
  random.shuffle(words)
  n = len(words)
  i1, i2 = int(0.70*n), int(0.85*n)
  splits = {"train":words[:i1], "val":words[i1:i2], "test":words[i2:]}

  for split, wl in splits.items():
    for w in tqdm(wl, desc=split):
      cls_dir = DATA_ROOT/split/w
      cls_dir.mkdir(parents=True, exist_ok=True)
      for k in range(VARIANTS_PER):
        stitch(w, cls_dir/f"{w}_{k}.png")

# 2. merge val/test into train
for old in ("val","test"):
  p = DATA_ROOT/old
  if p.exists():
    for cls in tqdm(list(p.iterdir()), desc=f"Merging {old}"):
      target = DATA_ROOT/"train"/cls.name
      target.mkdir(parents=True, exist_ok=True)
      for img in cls.glob("*.png"):
        shutil.move(img, target/img.name)
      shutil.rmtree(p)

# 3. create final test/ split (80/20)
print("➤ Splitting 80/20 into final train & test …")
test_dir.mkdir(parents=True, exist_ok=True)

for cls in tqdm(list(train_dir.iterdir()), desc="80/20"):
  imgs = list(cls.glob("*.png"))
  n_move = math.floor(len(imgs)*FINAL_TEST_FRAC)
  random.shuffle(imgs)
  tgt_cls = test_dir/cls.name
  tgt_cls.mkdir(parents=True, exist_ok=True)
  for img in imgs[:n_move]:
    shutil.move(img, tgt_cls/img.name)

# stats
print("\nDataset ready.")
print("Class folders  :", class_dir_count(train_dir), "(train) |",
  class_dir_count(test_dir), "(test)")
print("Image counts   :", sum(1 for _ in train_dir.rglob("*.png")), "(train) |",
  sum(1 for _ in test_dir.rglob("*.png")),  "(test)")

# **3. Load & Freeze the Single-Char Model**

In [ ]:
base_model = tf.keras.models.load_model(f"{BASE_PATH}/char_cnn_ckpt_best.keras")
base_model.trainable = False #freeze weights initially

print("Base model frozen — params:", base_model.count_params())

# **4. Data Loading & Splitting**

In [ ]:
# Train dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  labels="inferred",
  label_mode="categorical",
  batch_size=BATCH,
  image_size=IMG_SHAPE,
  color_mode="grayscale",
  validation_split=0.2,
  subset="training",
  seed=42
)

# Val dataset
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  train_dir,
  labels="inferred",
  label_mode="categorical",
  batch_size=BATCH,
  image_size=IMG_SHAPE,
  color_mode="grayscale",
  validation_split=0.2,
  subset="validation",
  seed=42
)

# Test dataset
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  test_dir,
  labels="inferred",
  label_mode="categorical",
  batch_size=BATCH,
  image_size=IMG_SHAPE,
  color_mode="grayscale",
  shuffle=False
)

# **5. Visual Sanity Check**

In [ ]:
# Utility to display examples from each set
def show_examples(ds, ds_name, num=5):
  # Take one batch
  for images, labels in ds.take(1):
    images = images.numpy()
    labels = labels.numpy()
    class_names = ds.class_names
    break

  plt.figure(figsize=(6,6))
  for i in range(num):
    ax = plt.subplot(3, 3, i+1)
    img = images[i].squeeze()  # shape: (H,W) since grayscale
    lbl = class_names[labels[i].argmax()]
    plt.imshow(img, cmap='gray')
    plt.title(f"{ds_name}: {lbl}")
    plt.axis('off')
  plt.tight_layout()
  plt.show()

# Display 5 examples from each split
show_examples(train_ds, "Train")
show_examples(val_ds, "Val")
show_examples(test_ds, "Test")

# **6. Build the Sliding-Window Model**

In [ ]:
def extract_patch(x, idx):
  start = idx * PATCH_W
  return x[:, :, start:start+PATCH_W, :] # (None, 64, 21, 1)

inputs = tf.keras.Input(shape=(IMG_H, IMG_W, 1))
logits = []

for i in range(3):
  patch = tf.keras.layers.Lambda(lambda z, i=i: extract_patch(z, i))(inputs)
  patch = tf.keras.layers.Resizing(IMG_H, IMG_H)(patch) # -> (64 x 64 x 1)
  # Re-use frozen base_model (shared weights)
  logits.append(base_model(patch)) # (None, 26)

concat = tf.keras.layers.Concatenate()(logits) # (None, 78)
# Hidden layer #1
h1 = tf.keras.layers.Dense(256, activation='relu')(concat)
h1 = tf.keras.layers.BatchNormalization()(h1)
h1 = tf.keras.layers.Dropout(0.4)(h1)
# Hidden layer #2
h1 = tf.keras.layers.Dense(256, activation='relu')(h1)
h1 = tf.keras.layers.Dropout(0.4)(h1)
outputs = tf.keras.layers.Dense(EXPECTED_CLASSES, activation='softmax')(h1)

word_model = tf.keras.Model(inputs, outputs)
word_model.compile(
  optimizer=tf.keras.optimizers.Adam(1e-3),
  loss='categorical_crossentropy',
  metrics=['accuracy']
)

# Print model summary
word_model.summary()

# **7. Callbacks**

In [10]:
callbacks = [
  # 1. Checkpoint
  ModelCheckpoint(CKPT_DIR, save_best_only=True, monitor='val_loss'),
  # 2. Early stopping
  EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True),
  # 3. Learning rate scheduler
  ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)
]

# **8. Train**

In [ ]:
# Train the model
history = word_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=20,
  callbacks=callbacks
)

In [ ]:
# (Optional: if accuracy is not great)
# Freeze all weights except the last N blocks
N = 1
# Un-freeze last 3 layers of base_model
for layer in base_model.layers[-N:]:
  layer.trainable = True

# Re-compile with lower LR
word_model.compile(
  optimizer=tf.keras.optimizers.Adam(1e-4),
  loss='categorical_crossentropy',
  metrics=['accuracy']
)

ft_history = word_model.fit(
  train_ds,
  validation_data=val_ds,
  initial_epoch=history.epoch[-1] + 1,
  epochs=history.epoch[-1] + 5,
  callbacks=callbacks
)

# **9. Evaluation**

In [ ]:
word_model = tf.keras.models.load_model(CKPT_DIR) # best checkpoint
test_loss, test_acc = word_model.evaluate(test_ds)
print(f"Test accuracy: {test_acc:.4f}")

# Util for plotting confusion matrix
def plot_confusion_matrix(cm, class_names, title="Confusion Matrix"):
  """
  Args:
      cm (np.ndarray): square confusion matrix
      class_names (List[str]): labels in the same order used to build cm
  """
  fig, ax = plt.subplots(figsize=(10, 9))
  im = ax.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
  ax.figure.colorbar(im, ax=ax, fraction=0.045)

  # axes & ticks
  ax.set(
    xticks=np.arange(len(class_names)),
    yticks=np.arange(len(class_names)),
    xticklabels=class_names,
    yticklabels=class_names,
    ylabel="True label",
    xlabel="Predicted label",
    title=title,
  )
  plt.setp(ax.get_xticklabels(), rotation=90, ha="center", va="center")

  # annotate cells
  thresh = cm.max() / 2.0
  for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
      ax.text(
        j, i, format(cm[i, j], "d"),
        ha="center", va="center",
        color="white" if cm[i, j] > thresh else "black",
        fontsize=8
      )

  fig.tight_layout()
  plt.show()

# Classification report
y_pred, y_true = [], []
for x, y in test_ds:
  y_pred.extend(np.argmax(word_model.predict(x), axis=1))
  y_true.extend(np.argmax(y.numpy(), axis=1))
print(classification_report(y_true, y_pred, target_names=train_ds.class_names))

# Confusion matrix heat-map (optional)
cm = confusion_matrix(y_true, y_pred)
plot_confusion_matrix(cm, train_ds.class_names, title="3-Letter Word Confusion Matrix")

# **10. Qualitative Error Analysis**

In [ ]:
# Plot a few misclassified 3-letter words
mis_idx = [i for i,(t,p) in enumerate(zip(y_true, y_pred)) if t != p]
show_examples(test_ds.unbatch().skip(mis_idx[0]), "Misclassified example")